# Example 6.13

In [1]:
import pathlib
import subprocess

import numpy as np
import pandas as pd
from collections import namedtuple
from IPython.display import display, Math
from scipy import linalg as la
from scipy import stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA

In [2]:
def load_data() -> pd.DataFrame:
    fpth = pathlib.Path(r'..\..\data\Table6.4.xlsx')
    df = pd.read_excel(fpth)
    df.rename(columns={'rate of extrusion': 'factor1',
                       'amount of additive': 'factor2',
                       'tear resistance': 'x1',
                       'gloss': 'x2',
                       'opacity': 'x3'}, inplace=True)
    
    df['factor1'] = df['factor1'].astype('category')
    df['factor2'] = df['factor2'].astype('category')

    return df

In [3]:
film_df = load_data()

In [4]:
# Using MANOVA from statsmodels, we get a Wilk's lambda value different than SAS or R.
manova = MANOVA.from_formula('x1 + x2 + x3 ~ C(factor1) * C(factor2)', data=film_df)
print(manova.mv_test(skip_intercept_test=True))

                 Multivariate linear model
                                                            
------------------------------------------------------------
       C(factor1)       Value  Num DF  Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda 0.4627 3.0000 14.0000  5.4184 0.0110
         Pillai's trace 0.5373 3.0000 14.0000  5.4184 0.0110
 Hotelling-Lawley trace 1.1611 3.0000 14.0000  5.4184 0.0110
    Roy's greatest root 1.1611 3.0000 14.0000  5.4184 0.0110
------------------------------------------------------------
                                                            
------------------------------------------------------------
       C(factor2)       Value  Num DF  Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda 0.8134 3.0000 14.0000  1.0707 0.3931
         Pillai's trace 0.1866 3.0000 14.0000  1.0707 0.3931
 Hotelling-Lawley trace 0.2294 3.0000 14.0

In [35]:
# The SS_sq terms in this output are the diagonal terms from from the SPP matrices on page 319.
# print("\nANOVA tables for each variable:")
# for dependent_var in ['x1', 'x2', 'x3']:
#     model = ols(f'{dependent_var} ~ factor1 * factor2', data=film_df).fit()
#     anova_table = sm.stats.anova_lm(model, typ=2)
#     print(f"\nANOVA table for {dependent_var}:\n", anova_table)

## Compute MANOVA using formulas on page 316

In [6]:
measure_cols = ['x1','x2','x3']
p = film_df.columns.size - 2
n = set(film_df.groupby(['factor1','factor2'],observed=False).size()).pop()

In [7]:
def compute_factors(df: pd.DataFrame, a_factor: str, measure_cols: list[str], n: int, p: int) -> np.ndarray:
    '''Compute the sum of squares and cross-products for individual factors.'''
    # Global mean.
    xbar = np.mean(film_df[measure_cols],axis=0).to_numpy()[:,np.newaxis]
    # Sub-group mean. Mean computed for each variable (measure) at each level of the factor.
    mean_vectors = df.groupby(a_factor,observed=False).mean([measure_cols])
    # Sub-group size.
    w = film_df[a_factor].unique().size

    result = np.zeros((p, p))
    for _, xbar_row in mean_vectors.iterrows():
        d = xbar_row.to_numpy()[:,np.newaxis] - xbar
        result += w*n*(d @ d.T)
    return result

In [8]:
def compute_interaction(df: pd.DataFrame, int_factors: list[str], measure_cols: list[str], n: int, p: str) -> np.ndarray:
    '''Compute the sum of squares and cross-products for the interaction term.'''
    # Global mean.
    xbar = np.mean(df[measure_cols],axis=0).to_numpy()[:,np.newaxis]
    # Sub-group mean. Mean computed for each variable (measure) at each level of the factor.
    mean_vectors = np.stack([df.groupby(a_factor,observed=False).mean([measure_cols]) for a_factor in int_factors])
    # Interaction mean.
    int_mean_vectors = df.groupby(int_factors,observed=False).mean([measure_cols])

    result = np.zeros((p, p))
    for idx, int_xbar_row in int_mean_vectors.iterrows():
        xbar_lk = int_xbar_row.to_numpy()[:,np.newaxis]
        xbar_l = mean_vectors[0,idx[0],:][:,np.newaxis]
        xbar_k = mean_vectors[1,idx[1],:][:,np.newaxis]
        d = xbar_lk - xbar_l - xbar_k + xbar
        result += n*(d @ d.T)
    return result

In [9]:
def compute_residual_error(df: pd.DataFrame, int_factors: list[str], measure_cols: list[str], p: str) -> np.ndarray:
    '''Compute the residual sum of squares and cross products.'''
    # Interaction mean.
    int_mean_vectors = df.groupby(int_factors,observed=False).mean([measure_cols])

    # Merge the interaction means onto the data.
    col_renames =  {c: '_'.join([c,'mean']) for c in int_mean_vectors}
    merged_df = df.merge(int_mean_vectors.rename(columns=col_renames).reset_index(),validate='m:1')

    # Create the difference values (obs - int_mean).
    difference_cols = list()
    for c in col_renames:
        diff_col = '_'.join([c,'d'])
        merged_df[diff_col] = merged_df[c] - merged_df[c + '_mean']
        difference_cols.append(diff_col)

    # Compute the outer products for the SS_{res}.
    result = np.zeros((p, p))
    for i, row in merged_df[difference_cols].iterrows():
        d = row.to_numpy()[:,np.newaxis]
        result += d @ d.T

    return result

In [10]:
def compute_total_corrected_ss(df: pd.DataFrame, measure_cols: list[str], p: str) -> np.ndarray:
    '''Compute the sum of squares and cross product total.'''
    df = df.copy()

    # Global mean.
    xbar = np.mean(df[measure_cols],axis=0)
    for c in np.mean(film_df[measure_cols],axis=0).index:
        df['_'.join([c,'mean'])] = np.mean(df[measure_cols],axis=0)[c]

    # Create the difference values (obs - global_mean).
    difference_cols = list()
    for c in measure_cols:
        diff_col = '_'.join([c,'d'])
        df[diff_col] = df[c] - df['_'.join([c,'mean'])]
        difference_cols.append(diff_col)

    # Compute the outer products for the SS_{cor}.
    result = np.zeros((p, p))
    for i, row in df[difference_cols].iterrows():
        d = row.to_numpy()[:,np.newaxis]
        result += d @ d.T

    return result
    

In [11]:
SSP_fac1 = compute_factors(film_df, 'factor1', measure_cols, n, p)
SSP_fac2 = compute_factors(film_df, 'factor2', measure_cols, n, p)
SSP_int = compute_interaction(film_df, ['factor1','factor2'], measure_cols, n, p)
SSP_res = compute_residual_error(film_df, ['factor1','factor2'], measure_cols, p)
SSP_cor = compute_total_corrected_ss(film_df, measure_cols, p)

In [12]:
g = film_df.factor1.unique().size
b = film_df.factor2.unique().size

In [13]:
display(Math(r'\begin{array}{lcc}'
             r'\hline \\'
             r'\text{Source of variation} & \text{SSP} & \text{Degrees of freedom} \\'
             r' &  &  \\'
             r'\hline \\'
             r'\text{Factor 1:} &'
             r' \left[ \begin{array}{rrr}'
             fr' {SSP_fac1[0,0]:.4f} & {SSP_fac1[0,1]:.4f} & {SSP_fac1[0,2]:.4f} \\'
             fr'                     & {SSP_fac1[1,1]:.4f} & {SSP_fac1[1,2]:.4f} \\'
             fr'                     &                     & {SSP_fac1[2,2]:.4f}'
             r' \end{array} \right] &'
             fr'g - 1 = {g} - 1 = {g - 1} \\ \\'
             r'\text{Factor 2:} &'
             r' \left[ \begin{array}{rrr}'
             fr' {SSP_fac2[0,0]:.4f} & {SSP_fac2[0,1]:.4f} & {SSP_fac2[0,2]:.4f} \\'
             fr'                     & {SSP_fac2[1,1]:.4f} & {SSP_fac2[1,2]:.4f} \\'
             fr'                     &                     & {SSP_fac2[2,2]:.4f}'
             r' \end{array} \right] &'
             fr'b - 1 = {b} - 1 = {b - 1} \\ \\'
             r'\text{Interaction} &'
             r' \left[ \begin{array}{rrr}'
             fr' {SSP_int[0,0]:.4f} & {SSP_int[0,1]:.4f} & {SSP_int[0,2]:.4f} \\'
             fr'                    & {SSP_int[1,1]:.4f} & {SSP_int[1,2]:.4f} \\'
             fr'                    &                    & {SSP_int[2,2]:.4f}'
             r' \end{array} \right] &'
             fr'(g - 1)(b - 1) = ({2} - 1)({b} - 1) = {(g-1)*(b-1)} \\ \\'
             r'\text{Residual} &'
             r' \left[ \begin{array}{rrr}'
             fr' {SSP_res[0,0]:.4f} & {SSP_res[0,1]:.4f} & {SSP_res[0,2]:.4f} \\'
             fr'                    & {SSP_res[1,1]:.4f} & {SSP_res[1,2]:.4f} \\'
             fr'                    &                    & {SSP_res[2,2]:.4f}'
             r' \end{array} \right] &'
             fr'gb(n - 1) = {g}({b})({n} - {1}) = {g*b*(n - 1)} \\ \\'
             r'\hline \\'
             r'\text{Total (corrected)} &'
             r' \left[ \begin{array}{rr}'
             fr' {SSP_cor[0,0]:.4f} & {SSP_cor[0,1]:.4f} & {SSP_cor[0,2]:.4f} \\'
             fr'                    & {SSP_cor[1,1]:.4f} & {SSP_cor[1,2]:.4f} \\'
             fr'                    &                    & {SSP_cor[2,2]:.4f}'
             r' \end{array} \right] &'
             f'gbn - 1 = {g}({b})({n}) - 1 = {g*b*n - 1}'
             r'\end{array}'
             ))

<IPython.core.display.Math object>

## Test for interaction

In [15]:
Lmbda_star_int = la.det(SSP_res) / la.det(SSP_int + SSP_res)
display(Math(r'\Lambda^{\star}'
             '='
             r'\frac{\left| \text{SSP}_{\text{res}} \right|}{\left| \text{SSP}_{\text{int}} + \text{SSP}_{\text{res}} \right|}'
             '='
             fr'\frac{{ {la.det(SSP_res):.4f} }}{{ {la.det(SSP_int + SSP_res):.4f} }}'
             '='
             f'{la.det(SSP_res) / la.det(SSP_int + SSP_res):.4f}'
             ))

<IPython.core.display.Math object>

In [16]:
F_int = ((1-Lmbda_star_int)/Lmbda_star_int)*((g*b*(n-1)-p+1)/2) / ((np.abs((g-1)*(b-1) - p) + 1)/2)
display(Math('F'
             '='
             r'\left( \frac{1 - \Lambda^{\star}}{\Lambda^{\star}} \right)'
             r'\frac{(gb(n-1) - p + 1)/2}{(|(g-1)(b-1) - p| + 1)/2}'
             '='
             fr'\left( \frac{{ 1 - {Lmbda_star_int:.4f} }}{{ {Lmbda_star_int:.4f} }} \right)'
             fr'\frac{{ ({g}({b})({n-1}) - {p} + 1) /2 }}{{ (|{g-1}({b-1}) - {p}| + 1)/2 }}'
             '='
             f'{F_int:.2f}'
             ))

<IPython.core.display.Math object>

In [17]:
v1_int = np.abs((g-1)*(b-1) - p) + 1
display(Math(r'\nu_{1}'
             '='
             r'\left| (g-1)(b-1) - p \right| + 1'
             '='
             fr'\left| {g-1}({b-1}) - {p} \right| + 1'
             '='
             f'{v1_int}'
             ))

<IPython.core.display.Math object>

In [18]:
v2_int = g*b*(n-1) - p + 1
display(Math(r'\nu_{2}'
             '='
             'gb(n - 1) - p + 1'
             '='
             fr'{g}({b})({n-1}) - {p} + 1'
             '='
             f'{v2_int}'
             ))

<IPython.core.display.Math object>

In [19]:
alpha = 0.05
f_crit_int = stats.f.ppf(1-alpha,v1_int,v2_int)
display(Math(r'F_{\nu_{1}, \nu_{2}}(\alpha)'
             '='
             fr'F_{{ {v1_int}, {v2_int} }}({alpha})'
             '='
             f'{f_crit_int:.2f}'
             ))

<IPython.core.display.Math object>

In [20]:
if F_int > f_crit_int:
    display(Math(fr'\text{{We have that }} F = {F_int:.2f} > F_{{\text{{crit}}}} = F_{{ {v1_int}, {v2_int} }} \left( {alpha} \right) = '
                 fr'{f_crit_int:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\gamma}_{11} = \bm{\gamma}_{12} = \bm{\gamma}_{21} = \bm{\gamma}_{22} = \textbf{0}'))
else:
    display(Math(fr'\text{{We have that }} F = {F_int:.2f} < F_{{\text{{crit}}}} = F_{{ {v1_int}, {v2_int} }} \left( {alpha} \right) = '
                fr'{f_crit_int:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\gamma}_{11} = \bm{\gamma}_{12} = \bm{\gamma}_{21} = \bm{\gamma}_{22} = \textbf{0}'))

<IPython.core.display.Math object>

## Test for factor 1 and factor 2 effects

In [21]:
Lmbda_star_fac1 = la.det(SSP_res) / la.det(SSP_fac1 + SSP_res)
display(Math(r'\Lambda_{1}^{\star}'
             '='
             r'\frac{\left| \text{SSP}_{\text{res}} \right|}{\left| \text{SSP}_{\text{fac1}} + \text{SSP}_{\text{res}} \right|}'
             '='
             fr'\frac{{ {la.det(SSP_res):.4f} }}{{ {la.det(SSP_fac1 + SSP_res):.4f} }}'
             '='
             f'{Lmbda_star_fac1:.4f}'
             ))

<IPython.core.display.Math object>

In [22]:
F_fac1 = ((1-Lmbda_star_fac1)/Lmbda_star_fac1)*((g*b*(n-1)-p+1)/2) / ((np.abs((g-1) - p) + 1)/2)
display(Math('F_{1}'
             '='
             r'\left( \frac{1 - \Lambda_{1}^{\star}}{\Lambda_{1}^{\star}} \right)'
             r'\frac{(gb(n-1) - p + 1)/2}{(|(g-1) - p| + 1)/2}'
             '='
             fr'\left( \frac{{ 1 - {Lmbda_star_fac1:.4f} }}{{ {Lmbda_star_fac1:.4f} }} \right)'
             fr'\frac{{ ({g}({b})({n-1}) - {p} + 1) /2 }}{{ (|{g-1} - {p}| + 1)/2 }}'
             '='
             f'{F_fac1:.2f}'
             ))

<IPython.core.display.Math object>

In [23]:
v1_fac1 = np.abs((g-1) - p) + 1
display(Math(r'\nu_{1}'
             '='
             '|(g-1) - p| + 1'
             '='
             f'|{g-1} - {p}| + 1'
             '='
             f'{v1_fac1}'
             ))

<IPython.core.display.Math object>

In [24]:
v2_fac1 = g*b*(n-1)-p+1
display(Math(r'\nu_{2}'
             '='
             'gb(n-1) - p + 1'
             '='
             f'{g*b*(n-1)} - {p} + 1'
             '='
             f'{v2_fac1}'
             ))

<IPython.core.display.Math object>

In [25]:
f_crit_fac1 = stats.f.ppf(1-alpha,v1_fac1,v2_fac1)
display(Math(r'F_{\nu_{1}, \nu_{2}}(\alpha)'
             '='
             fr'F_{{ {v1_fac1}, {v2_fac1} }}({alpha})'
             '='
             f'{f_crit_fac1:.2f}'
             ))

<IPython.core.display.Math object>

In [26]:
if F_fac1 > f_crit_fac1:
    display(Math(fr'\text{{We have that }} F_{{1}} = {F_fac1:.2f} > F_{{\text{{crit}}}} = F_{{ {v1_fac1}, {v2_fac1} }} \left( {alpha} \right) = '
                 fr'{f_crit_fac1:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\tau}_{1} = \bm{\tau}_{2} = \textbf{0}'))
else:
    display(Math(fr'\text{{We have that }} F_{{1}} = {F_fac1:.2f} < F_{{\text{{crit}}}} = F_{{ {v1_fac1}, {v2_fac1} }} \left( {alpha} \right) = '
                fr'{f_crit_fac1:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\tau}_{1} = \bm{\tau}_{2} = \textbf{0}'))

<IPython.core.display.Math object>

In [27]:
Lmbda_star_fac2 = la.det(SSP_res) / la.det(SSP_fac2 + SSP_res)
display(Math(r'\Lambda_{2}^{\star}'
             '='
             r'\frac{\left| \text{SSP}_{\text{res}} \right|}{\left| \text{SSP}_{\text{fac2}} + \text{SSP}_{\text{res}} \right|}'
             '='
             fr'\frac{{ {la.det(SSP_res):.4f} }}{{ {la.det(SSP_fac2 + SSP_res):.4f} }}'
             '='
             f'{Lmbda_star_fac2:.4f}'
             ))

<IPython.core.display.Math object>

In [28]:
F_fac2 = ((1-Lmbda_star_fac2)/Lmbda_star_fac2)*((g*b*(n-1)-p+1)/2) / ((np.abs((b-1) - p) + 1)/2)
display(Math('F_{2}'
             '='
             r'\left( \frac{1 - \Lambda_{2}^{\star}}{\Lambda_{2}^{\star}} \right)'
             r'\frac{(gb(n-1) - p + 1)/2}{(|(b-1) - p| + 1)/2}'
             '='
             fr'\left( \frac{{ 1 - {Lmbda_star_fac2:.4f} }}{{ {Lmbda_star_fac2:.4f} }} \right)'
             fr'\frac{{ ({g}({b})({n-1}) - {p} + 1) /2 }}{{ (|{b-1} - {p}| + 1)/2 }}'
             '='
             f'{F_fac2:.2f}'
             ))

<IPython.core.display.Math object>

In [29]:
v1_fac2 = np.abs((b-1) - p) + 1
display(Math(r'\nu_{1}'
             '='
             '|(b-1) - p| + 1'
             '='
             f'|{b-1} - {p}| + 1'
             '='
             f'{v1_fac2}'
             ))

<IPython.core.display.Math object>

In [30]:
v2_fac2 = g*b*(n-1)-p+1
display(Math(r'\nu_{2}'
             '='
             'gb(n-1) - p + 1'
             '='
             f'{g*b*(n-1)} - {p} + 1'
             '='
             f'{v2_fac2}'
             ))

<IPython.core.display.Math object>

In [31]:
f_crit_fac2 = stats.f.ppf(1-alpha,v1_fac2,v2_fac2)
display(Math(r'F_{\nu_{1}, \nu_{2}}(\alpha)'
             '='
             fr'F_{{ {v1_fac2}, {v2_fac2} }}({alpha})'
             '='
             f'{f_crit_fac2:.2f}'
             ))

<IPython.core.display.Math object>

In [32]:
if F_fac2 > f_crit_fac2:
    display(Math(fr'\text{{We have that }} F_{{2}} = {F_fac2:.2f} > F_{{\text{{crit}}}} = F_{{ {v1_fac2}, {v2_fac2} }} \left( {alpha} \right) = '
                 fr'{f_crit_fac2:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\beta}_{1} = \bm{\beta}_{2} = \textbf{0}'))
else:
    display(Math(fr'\text{{We have that }} F_{{1}} = {F_fac2:.2f} < F_{{\text{{crit}}}} = F_{{ {v1_fac2}, {v2_fac2} }} \left( {alpha} \right) = '
                fr'{f_crit_fac2:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                r'\bm{\beta}_{1} = \bm{\beta}_{2} = \textbf{0}'))

<IPython.core.display.Math object>

## MANOVA Wilk's lambda results from R

Using manova in R we get a matching Wilk's lambda.

In [33]:
result = subprocess.run(['Rscript', r'..\..\r\chapter-6\Example6-13.R'], capture_output=True, text=True)
print(result.stdout)

                Df   Wilks approx F num Df den Df   Pr(>F)   
factor1          1 0.38186   7.5543      3     14 0.003034 **
factor2          1 0.52303   4.2556      3     14 0.024745 * 
factor1:factor2  1 0.77711   1.3385      3     14 0.301782   
Residuals       16                                           
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

